In [1]:
YOUR_NAME = 'sara'

AWS_PROFILE = 'cities'

In [2]:
%load_ext autoreload

In [3]:
MAIN_PATH = "s3://wri-cities-sandbox/identifyingLandSubdivisions/data"
INPUT_PATH = f'{MAIN_PATH}/input'
CITY_INFO_PATH = f'{INPUT_PATH}/city_info'
EXTENTS_PATH = f'{CITY_INFO_PATH}/extents'
BUILDINGS_PATH = f'{INPUT_PATH}/buildings'
BLOCKS_PATH = f'{INPUT_PATH}/blocks'
ROADS_PATH = f'{INPUT_PATH}/roads'
INTERSECTIONS_PATH = f'{INPUT_PATH}/intersections'
GRIDS_PATH = f'{INPUT_PATH}/city_info/grids'
SEARCH_BUFFER_PATH = f'{INPUT_PATH}/city_info/search_buffers'
OUTPUT_PATH = f'{MAIN_PATH}/output'
OUTPUT_PATH_CSV = f'{OUTPUT_PATH}/csv'
OUTPUT_PATH_RASTER = f'{OUTPUT_PATH}/raster'
OUTPUT_PATH_PNG = f'{OUTPUT_PATH}/png'
OUTPUT_PATH_RAW = f'{OUTPUT_PATH}/raw_results'

In [4]:
# Check s3 connection using AWS_PROFILE=CitiesUserPermissionSet profile 
import boto3
boto3.setup_default_session(profile_name='cities')
session = boto3.Session(profile_name=AWS_PROFILE)
s3 = session.client('s3')

# export CitiesUserPermissionSet profile to use in the next cells
import os
os.environ['AWS_PROFILE'] = AWS_PROFILE

s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'CRQSQW00SR1BG8NN',
  'HostId': 'cY/PJ9gaHS6N4UC4Md5ISXYCFPR+4algGL4hZ3ZM7hw1ycIAoALKg30GqxDWhaxanoqJTL6nypSD3ai0VatOORbrBtWGf/my',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'cY/PJ9gaHS6N4UC4Md5ISXYCFPR+4algGL4hZ3ZM7hw1ycIAoALKg30GqxDWhaxanoqJTL6nypSD3ai0VatOORbrBtWGf/my',
   'x-amz-request-id': 'CRQSQW00SR1BG8NN',
   'date': 'Wed, 17 Dec 2025 17:11:39 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'aft-sandbox-540362055257',
   'CreationDate': datetime.datetime(2022, 9, 13, 15, 12, 20, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::aft-sandbox-540362055257'},
  {'Name': 'amplify-citiesindicatorsapi-dev-10508-deployment',
   'CreationDate': datetime.datetime(2023, 8, 30, 5, 5, 13, tzinfo=tzutc()),
   'BucketArn': 'arn:aws:s3:::amplify-citiesindicatorsapi-dev-10508-deployment'},
  {'Name': 'cities-heat',
   'CreationDate': datetime.date

In [5]:
# START COILED CLIENT.

import coiled

cluster = coiled.Cluster(
    workspace="wri-cities-data",
    name=f'ils-{YOUR_NAME}',
    region="us-west-2",
    arm=True,
    worker_vm_types="r8g.xlarge",
    spot_policy="spot",
    n_workers=10,
    package_sync_ignore=["pyspark", "pypandoc",
                         "boto3",
                        "botocore",
                        "aiobotocore",
                        "urllib3"]
)
client = cluster.get_client()

print(f"Started a new Dask client on Coiled. Dashboard is available at {client.dashboard_link}")

[2025-12-17 12:11:40,116][INFO    ][coiled] Fetching latest package priorities...
[2025-12-17 12:11:40,118][INFO    ][coiled.package_sync] Resolving your local subdivisions2 Python environment...
[2025-12-17 12:11:40,571][INFO    ][coiled.package_sync] Scanning 443 conda packages...
[2025-12-17 12:11:40,575][INFO    ][coiled.package_sync] Scanning 258 python packages...
[2025-12-17 12:11:41,013][INFO    ][coiled.software_utils] No username or password found for https://conda.anaconda.org/conda-forge
[2025-12-17 12:11:41,533][INFO    ][coiled] Running pip check...
[2025-12-17 12:11:41,985][INFO    ][coiled] Validating environment...
[2025-12-17 12:11:45,412][INFO    ][coiled] Creating wheel for ~/Documents/Identifying Land Subdivisions/identifyingLandSubdivisions...
[2025-12-17 12:11:45,493][INFO    ][coiled] Creating wheel for /opt/spark-2.2.0/python...
[2025-12-17 12:11:45,644][WARNING ][coiled.package_sync] Package - debugpy, debugpy~=1.8.16 has no install candidate for Python 3.12 l

Started a new Dask client on Coiled. Dashboard is available at https://cluster-kzocb.dask.host/nEfax2ubx4pfQ5QR/status


/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/client.py:1590: VersionMismatchWarning: Mismatched versions found

+---------+--------+-----------+---------+
| Package | Client | Scheduler | Workers |
+---------+--------+-----------+---------+
| lz4     | 4.4.4  | 4.4.5     | 4.4.5   |
+---------+--------+-----------+---------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


2025-12-17 12:32:07,625 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2025-12-17 12:34:20,413 - distributed.deploy.cluster - WARNING - Failed to sync cluster info multiple times - perhaps there's a connection issue? Error:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/utils.py", line 1923, in wait_for
    return await fut
           ^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/distributed/comm/tcp.py", line 547, in connect
    stream = await self.client.connect(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/subdivisions2/lib/python3.12/site-packages/tornado/tcpclient.py", line 279, in connect
    af, addr, stream = await connector.start(connect_timeout=timeout)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
asyncio.exceptions.CancelledError

The above exception was the direct cause 

In [6]:
import dask_geopandas as dgpd
import dask.dataframe as dd
import pandas as pd
from dask import delayed, compute, visualize
import geopandas as gpd
from dask.diagnostics import ProgressBar
from shapely.geometry import MultiLineString, LineString, Point
from shapely.ops import polygonize, nearest_points
#from shapely.geometry import Polygon, LineString, Point, MultiPolygon, MultiLineString, GeometryCollection
from scipy.optimize import fminbound, minimize
#from unused_code.metrics_groupby import metrics
from dask import delayed
import dask_geopandas as dgpd
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkb
from scipy.stats import entropy
import time
from dask import compute

from pre_processing import *
from auxiliary_functions import *
from standardize_metrics import *
from metrics_calculation import *

import time

YOUR_NAME = 'sara'

In [ ]:
# CALCULATE BUILDING DISTANCES TO ROADS AND PRODUCE BLOCKS.

start_time = time.time()  # Start the timer

%autoreload
#from main import *

def preprocess_all_cities(city_list):
    delayed_jobs = []
    delayed_jobs = [delayed(calculate_building_distances_to_roads)(city) for city in city_list]
    #delayed_jobs.append([delayed(produce_blocks)(city) for city in city_list])
    #delayed_jobs.append([calculate_building_distances_to_roads(city) for city in city_list])
    delayed_jobs.append([produce_azimuths(city,YOUR_NAME) for city in city_list])
    #delayed_jobs.append([produce_blocks(city,YOUR_NAME,grid_size) for city in city_list])
    results = compute(*delayed_jobs)
    return results

cities = ["Accra", "Abidjan", "Bamako", "Bogota", "Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"]
preprocess_all_cities(cities) 


end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")

In [ ]:
start_time = time.time()  # Start the timer

cities = ["Accra", "Abidjan", "Bamako", "Bogota", "Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu",

cities = [city.replace(' ', '_') for city in cities]

tasks = []
for city in cities:
    tasks.append(building_and_intersection_metrics(city,YOUR_NAME))
    #tasks.append(building_distance_metrics(city, YOUR_NAME))
    #tasks.append(compute_m6_m7(city, YOUR_NAME))
    #tasks.append(metrics_roads_intersections(city,YOUR_NAME))
    #tasks.append(metrics_k_blocks(city,YOUR_NAME))

results = compute(*tasks)

end_time = time.time()  # End the timer
elapsed_time = end_time - start_time

print(f"Tasks completed in {elapsed_time:.2f} seconds.")



In [7]:
from dask import delayed, compute
from post_processing_auxiliaries import *

cities = ["Accra", "Abidjan", "Bamako", "Bogota", "Belo_Horizonte", "Campinas", "Cape_Town", "Lagos",  "Nairobi", "Luanda"] #, Maputo, "Mogadishu",

# build delayed tasks
tasks = [delayed(post_process_city)(c, YOUR_NAME) for c in cities]

# run them in parallel (uses your existing Dask Client / Coiled cluster)
results = compute(*tasks)
